In [2]:
#Kernal: Python[conda env:rpy2]

import pickle
import anndata2ri
import scanpy
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

#This activates the object conversion.
anndata2ri.activate()
%load_ext rpy2.ipython

/tmp/ipykernel_3871449/834370561.py:11: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [3]:
#Import needed R libraries.
seurat = importr('Seurat')
base = importr('base')


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In [4]:
#Read Seurat object into R.
robjects.r('seurat_object = readRDS("/scratch/smallapragada/run2/run2_airway_2024_08_15.rds")')

<rpy2.robjects.methods.RS4 object at 0x1555433a2d00> [RTYPES.S4SXP]
R classes: ('Seurat',)

In [6]:
#Merge layers because Seurat object versioning is wild. Convert Seurat object to Anndata.
#robjects.r('seurat_object = JoinLayers(seurat_object)')
adata = robjects.r('as.SingleCellExperiment(seurat_object)')

/opt/conda/envs/rpy2/lib/python3.10/site-packages/anndata2ri/_r2py.py:132: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return AnnData(exprs, obs, var, uns, obsm, layers=layers)


In [7]:
#Check to make sure conversion worked and possibly cast counts to array if necessary.
adata.X = adata.X.toarray()

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [8]:
adata.X

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 3., 0., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 3., 0.]], dtype=float32)

In [7]:
del adata.layers

In [9]:
#Write anndata to pickle file. Next move to the clustering script.
with open("merged_spatial_filtered_airway", "wb") as adata_object:
    pickle.dump(adata, adata_object)

In [10]:
adata

AnnData object with n_obs × n_vars = 106060 × 343
    obs: 'X', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'percent_negcodes', 'percent_negprobes', 'Sample', 'Slide', 'Timepoint', 'Phenotype', 'adj_x_centroid', 'adj_y_centroid', 'ident', 'leiden_brl_0.25', 'leiden_brl_0.5', 'leiden_brl_0.75', 'leiden_brl_1.0'
    obsm: 'SP', 'X_pca', 'X_umap'
    layers: 'logcounts', 'scaledata'